# Agent Tools

**Day 2A — Google ADK | Local VS Code**

You’ll build:
- Custom function tools
- Agents as tools
- Multi-tool agents

---
**No Kaggle. Just code.**

### Load API Key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Add GOOGLE_API_KEY to .env in project root!")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

print("Gemini API key loaded!")

Gemini API key loaded!


### Import ADK

In [2]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("ADK ready!")

ADK ready!


### Helper Function

In [3]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


### Retry Config

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Retry config set.")

✅ Retry config set.


## Why Agents Need Tools?

Without tools: Frozen knowledge, no actions.

With tools: Connect to world, get things done.

### Define Fee Function

In [5]:
def get_fee_for_payment_method(payment_method: str) -> dict:
    """Get the transaction fee percentage for the given payment method.
    
    Args:
        payment_method (str): The payment method to look up. Must be one of: 'Credit Card', 'Debit Card', 'Bank Transfer'.
    
    Returns:
        dict: A dictionary containing the status and fee percentage or error message.
    """
    
    fees = {
        "Credit Card": 0.03,
        "Debit Card": 0.02,
        "Bank Transfer": 0.01
    }
    
    if payment_method in fees:
        return {
            "status": "success",
            "fee_percentage": fees[payment_method]
        }
    else:
        return {
            "status": "error",
            "message": f"Invalid payment method: {payment_method}. Valid options are: 'Credit Card', 'Debit Card', 'Bank Transfer'."
        }

print("✅ get_fee_for_payment_method defined.")

✅ get_fee_for_payment_method defined.


### Fee Tool

In [6]:
from google.adk.tools import FunctionTool

get_fee_for_payment_method = FunctionTool(get_fee_for_payment_method)

print("✅ Fee tool created.")

✅ Fee tool created.


### Define Exchange Function

In [7]:
def get_exchange_rate(source_currency: str, target_currency: str) -> dict:
    """Get the exchange rate for converting source_currency to target_currency.
    
    Args:
        source_currency (str): The source currency code (e.g., 'USD').
        target_currency (str): The target currency code (e.g., 'INR').
    
    Returns:
        dict: A dictionary containing the status and exchange rate or error message.
    """
    
    # This is a mock implementation. In production, integrate with a real exchange rate API.
    rates = {
        ("USD", "INR"): 83.5,
        ("USD", "EUR"): 0.92,
        ("EUR", "USD"): 1.09
    }
    
    key = (source_currency.upper(), target_currency.upper())
    reverse_key = (key[1], key[0])
    
    if key in rates:
        return {
            "status": "success",
            "exchange_rate": rates[key]
        }
    elif reverse_key in rates:
        return {
            "status": "success",
            "exchange_rate": 1 / rates[reverse_key]
        }
    else:
        return {
            "status": "error",
            "message": f"Exchange rate not available for {source_currency} to {target_currency}."
        }

print("✅ get_exchange_rate defined.")

✅ get_exchange_rate defined.


### Exchange Tool

In [8]:
get_exchange_rate = FunctionTool(get_exchange_rate)

print("✅ Exchange tool created.")

✅ Exchange tool created.


### Currency Agent

In [9]:
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

For any currency conversion request:

  1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
  2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
  3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
  4. Calculate Final Amount: Perform the calculation for the final converted amount using this formula:
     a. Calculate fee amount: original_amount * fee_percentage
     b. Calculate amount after fee: original_amount - fee_amount
     c. Calculate converted amount: amount_after_fee * exchange_rate
  5. Provide Detailed Breakdown: In your summary, you must:
      * State the final converted amount.
      * Explain how the result was calculated, including:
          * The fee percentage and the fee amount in the original currency.
          * The amount remaining after deducting the fee.
          * The exchange rate applied.
   """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate
    ],
)

print("✅ Currency agent created.")

✅ Currency agent created.


### Runner for Currency

In [10]:
currency_runner = InMemoryRunner(agent=currency_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".


### Run Currency Query

In [11]:
response = await currency_runner.run_debug(
    "Convert 1,000 USD to INR using a Credit Card. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,000 USD to INR using a Credit Card. Show me the precise calculation.


currency_agent > The fee for using a Credit Card is 3%.

The exchange rate from USD to INR is 83.5.

Here's the calculation:

1.  **Fee amount:** 1,000 USD * 3% = 30 USD
2.  **Amount after fee:** 1,000 USD - 30 USD = 970 USD
3.  **Converted amount:** 970 USD * 83.5 INR/USD = 80,995 INR

So, 1,000 USD is converted to 80,995 INR when using a Credit Card.


## Enhanced: Use Agents as Tools

Delegate calculations to specialist agent.

### Calculation Agent

In [12]:
calculation_agent = LlmAgent(
    name="calculation_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a precise calculation agent. Your ONLY job is to generate and execute Python code for the EXACT calculation requested.
    
    Rules:
    1.  You MUST generate Python code that EXACTLY matches the requested calculation.
    2.  The code MUST be a single expression or simple script that computes the result.
    3.  Use the code executor tool to run your generated code.
    4.  The generated Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),
)

print("✅ Calculation agent created.")

✅ Calculation agent created.


### Enhanced Currency Agent

In [13]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),
    ],
)

print("✅ Enhanced currency agent created.")

✅ Enhanced currency agent created.


### Runner for Enhanced

In [14]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".


### Run Enhanced Query

In [15]:
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".


enhanced_currency_agent > The final converted amount is 103,331.25 INR.

Here's the breakdown of the calculation:
*   The transaction fee for a Bank Transfer is 1% (0.01).
*   The fee amount is 12.50 USD (1,250 USD \* 0.01).
*   After deducting the fee, the amount remaining is 1,237.50 USD (1,250 USD - 12.50 USD).
*   The exchange rate applied is 83.5 INR per USD.
*   The final converted amount is 103,331.25 INR (1,237.50 USD \* 83.5).


### Show Results

In [16]:
show_python_code_and_result(response)

Generated Python Response >>  This is a great start! It looks like you've already calculated most of what's needed. However, there's a small typo: "mount_after_fee" should be "amount_after_fee".

Once that's corrected, the code should run perfectly. I'll use the corrected code to perform the calculation for you.


The calculations are as follows:

Fee Percentage: 0.01
Fee Amount (USD): 12.5
Amount after fee (USD): 1237.5
Exchange Rate: 83.5
Converted Amount (INR): 103331.25


## Agent Tools vs Sub-Agents

Agent Tools: Call & get response back (delegation)

Sub-Agents: Full handoff (transfer control)

## ADK Tool Types

Custom: Function, Long-Running, Agent, MCP, OpenAPI

Built-in: Gemini (search, code exec), Cloud, Third-party

### Congrats

## DONE!

You built:
- Function tools
- Agents as tools
- Multi-tool agents